In [108]:
import pandas as pd
import geopandas as gpd
import glob
import math
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [109]:
# created fields should be in lower case to separate from Caltrans
# could apply addition framework
# propose deleting any intermediate variables (e.g., party_sev_PARTY)

#### Isolate Appropriate Crashes
Crashes occuring between January 1st 2020 and December 31st 2024

In [110]:
# for testing
df_crash = pd.read_excel(r"\\kittelson.com\fs\H_Projects\27\27381 - Caltrans District Plans\Task 3 Methodology\Data\data\TSN_Data\Crash_Data\CA AX 2022 v6.26.2023.xlsx", nrows = 100)

#### Crash Severity by Party
Assign severity for each party. Pre-step for assigning crash party.

In [111]:
# Function to apply logis to each party to calculate severity
def party_sev(party_type, crash_severity, major_injury, minor_injury, possible_injury, injury_code, occupants_killed, occupants_injured):
    if type(party_type) != str:
        party_sev = 99
    elif (injury_code == 'K') | (occupants_killed > 0):
        party_sev = 1
    elif (major_injury > 0):
        party_sev = 2
    elif (minor_injury > 0):
        party_sev = 3
    elif (possible_injury > 0):
        party_sev = 4
    elif (occupants_injured > 0): # crashes that report occupand injured but none by other variables (small number of issues)
        if crash_severity in [1,2]:
            party_sev = 2
        elif crash_severity in [3,4]:
            party_sev = crash_severity
    else:
        party_sev = 5 # Note I am using 5 instead of 0 to allow for calculating max severity on a continuous scale.
    return party_sev

In [112]:
x = 1

while x < 10:
    party = str(x)

    df_crash['party_sev_' +str(x)] = df_crash.apply(lambda row: party_sev(row['PARTY_TYPE_' + party], row['SEVERITY_LEVEL'], row['MAJOR_INJURY_' + party], 
                                                row['MINOR_INJURY_' + party], row['POSSIBLE_INJURY_' + party], row['INJURY_CODE_' + party], 
                                                row['OCCUPANTS_KILLED_' + party], row['OCCUPANTS_INJURED_' + party]), axis=1)
    x = x + 1

#### Singular Crash Mode
Crash mode based on most severely injured party and then if tie by agreed logic in methodology.

In [113]:
# Identify for each crash the maximum severity based on the party severity variables.
df_crash['max_sev'] = df_crash[df_crash.columns[df_crash.columns.str[:-1] == "party_sev_"]].min(axis = 1)

In [114]:
def party_on_sev(row):
    x = 1
    parties = []
    while x < 10:
        if row['party_sev_' +str(x)] == row['max_sev']:
            parties.append(str(row['PARTY_TYPE_' +str(x)]))

        x = x + 1

    party_det = 'other / unknown'

    # Go through options from least to most dominant in logic such that if most dominant applied
    for x in parties:
        if x in ['A','B','D','E','F','G','2','3','4','5','6','H','I','J','K','M','Q']:
            party_det = 'motorist'

    for x in parties:
        if x in ['C','R']:
            party_det = 'motorcyclist'
    
    for x in parties:
        if x in ['V']:
            party_det = 'dismounted_pedestrian'
            
    for x in parties:
        if x in ['L']:
            party_det = 'bicyclist'

    for x in parties:
        if x in ['U']:
            party_det = 'pedestrian'

    return party_det

In [115]:
df_crash['crash_mode'] = df_crash.apply(party_on_sev, axis=1)

#### Crash Flags
Binary variables to identify presence of party type in crash.

In [116]:
party_fields = df_crash.columns[df_crash.columns.str[:-1] == "PARTY_TYPE_"]

In [117]:
def add_flags_mode(row):
    ped = 0
    bike = 0
    dped = 0
    mcycle = 0
    truck = 0
    animal = 0

    for x in party_fields:
        if row[x] in ['U']:
            ped = 1
        if row[x] in ['L']:
            bike = 1
        if row[x] in ['V']:
            dped = 1
        if row[x] in ['C','R']:
            mcycle = 1
        if str(row[x]) in ['F','G','2','3','4','5','6']:
            truck = 1
        if row[x] in ['W','X','Z']:
            animal = 1

    return ped, bike, dped, mcycle, truck, animal

In [118]:
output = df_crash.apply(add_flags_mode, axis=1)

In [119]:
df_crash[['f_ped','f_bike','f_dped','f_mcycle','f_truck','f_animal']] = pd.DataFrame(output.tolist())

In [120]:
df_crash.head()

,D,C,R,R_SFX,PM_PFX,PM,PM_SFX,ACC_ID,ACC_PLACEMENT_ID,ACC_COLL_TYPE_CODE,...,party_sev_8,party_sev_9,max_sev,crash_mode,f_ped,f_bike,f_dped,f_mcycle,f_truck,f_animal
0,12,ORA,1,NaN,R,0.180,NaN,5358438,2735672,C,...,99,99,4,motorist,0,0,0,0,0,0
1,12,ORA,1,NaN,R,0.204,NaN,5353056,233446,D,...,99,99,5,motorist,0,0,0,0,0,0
2,12,ORA,1,NaN,R,0.204,NaN,5443424,233446,D,...,99,99,5,motorist,0,0,0,0,0,0
3,12,ORA,1,NaN,R,0.280,NaN,5385364,327062,B,...,99,99,5,motorist,0,0,0,0,0,0
4,12,ORA,1,NaN,R,0.290,NaN,5402394,2402929,B,...,99,99,5,motorist,0,0,0,0,0,0


#### Location-Movement Crassified Field
Methodology based on guidance from Safe Streets.

#### Analysis Flag
Flag field to identify if crash is for consideration by Kittelson / F&P or Safe Streets. \
Uses singular crash mode to create binary. To be combined with roadway data to group into one of four analyses.

#### Caltrans Focus Crash Types
Binary variables identifying if crash should be considered for risk ratio analysis of focus crash types.

In [ ]:
# Pedestrian Crash
# Use pedestrian mode flag

In [ ]:
# Bicycle Crash
# Use bicycle mode flag

In [ ]:
# Impaired
# Fields need to be confirmed with Caltrans
# Use established logic

In [ ]:
# Aggressive Driving
# Fields need to be confirmed with Caltrans
# Use established logic

In [ ]:
# Lane Departure
# Use established logic

In [ ]:
# Intersections
# Use field in data